In [7]:
# image features를 .pkl 파일에 저장한다. 

In [6]:

# 1. 파일 경로 불러오기
import os
data_dir="/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769"

all_files = [os.path.join(data_dir, path) for path in os.listdir(data_dir) if path[0] != "." and path.endswith(".jpg")] # 이미지 파일 경로

print(len(all_files))
all_files=sorted(all_files)
print(all_files)



727
['/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_0.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_1.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_10.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_100.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_101.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_102.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_103.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_104.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_105.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_106.jpg', '/home/gpuadmin/dev/Trajectory

In [9]:
# 2. test : 한 개의 이미지로부터 불러오기
import torch
import torch.nn as nn
class conv2DBatchNormRelu(nn.Module):
    def __init__(self, in_channels, n_filters, k_size,  stride, padding, bias=True, dilation=1):
        super(conv2DBatchNormRelu, self).__init__()

        self.cbr_unit = nn.Sequential(
            nn.Conv2d(
                int(in_channels), # 입력 채널의 수, rgb의 경우 3
                      int(n_filters), # 필터의 수
                      kernel_size=k_size, # 커널의 크기
                        padding=padding, 
                        stride=stride, 
                        bias=bias, # bias (default=True) : 편향 사용의 여부
                        dilation=dilation # 딜레이션 레이트 : default = 1
                ),
            nn.BatchNorm2d(int(n_filters)), # 2차원 이미지 데이터에 대한 배치 정규화
            nn.ReLU(inplace=True),
                                 ) 

    def forward(self, inputs):
        outputs = self.cbr_unit(inputs)
        return outputs


class SemanticImageEncoder(nn.Module):
    '''
    Tianyang:
    Simple Convolutional Encoder for semantic map images
    Input size: flexible
    Output size: 1/2 input
    '''

    def __init__(self, in_channels = 3, out_channels = 32):
        super(SemanticImageEncoder, self).__init__()

        self._encoder = nn.Sequential(
            conv2DBatchNormRelu(in_channels = in_channels, # 입력 채널 개수, RGB일 때 : 3
                                n_filters = 16, # 출력 채널 개수
                                k_size = 3,     # 컨볼루션 필터 크기 : (3,3) 
                                stride = 1,         # 필터 이동 간격 (default=1)
                                padding = 1),       # padding :(default=0) 
            
            conv2DBatchNormRelu(in_channels = 16, n_filters = 16, \
                k_size = 4,  stride = 1, padding = 2),
            
            nn.MaxPool2d((2, 2), stride=(2, 2)),

            conv2DBatchNormRelu(in_channels = 16, n_filters = out_channels, \
                k_size = 5,  stride = 1, padding = 2),
            )

    def forward(self, input):
        # encoded = self._encoder(input.type(torch.cuda.FloatTensor))
        encoded = self._encoder(input.type(torch.FloatTensor))
        return encoded

In [10]:
image_channels=3
embed_dim_image=32
image_encoder = SemanticImageEncoder(in_channels = image_channels, out_channels = embed_dim_image)

In [11]:

train_path = "/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/"

val_path = "/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/770/"

In [12]:
from typing import Tuple
class CreateArg():
    def __init__(self):
        # Dataset options
        self.dataset_name = 'waterloo'
        self.delim = '\t'
        self.loader_num_workers = 4
        self.obs_len = 8
        self.pred_len = 8
        self.skip = 1
        # Optimization
        self.batch_size = 4                 #### batch_size test !!!
        self.num_iterations = 1000 # 10000
        self.num_epochs = 50 # 200                     
        # Model Options
        self.embedding_dim = 64
        self.num_layers = 1
        self.dropout = 0.0
        self.batch_norm = 0 
        self.mlp_dim = 1024
        # self.default_backbone= 'resnet18'
                                    
        # Generator Options
        self.encoder_h_dim_g = 64
        self.decoder_h_dim_g = 128
        self.noise_dim : Tuple[int] = (0, 0) # default=None # type=int_tuple
        self.noise_type = 'gaussian'
        self.noise_mix_type = 'ped'
        self.clipping_threshold_g = 0 # type=float
        self.g_learning_rate = 5e-4 # type=float 
        self.g_steps = 1

        # Pooling Options
        self.pooling_type = 'pool_net' 
        self.pool_every_timestep = 1 # type=bool_flag

        # Pool Net Option
        self.bottleneck_dim = 1024 # type=int

        # Social Pooling Options
        self.neighborhood_size = 1024 # type=float
        self.grid_size = 8 # type=int

        # Discriminator Options
        self.d_type = 'local' # type=str
        self.encoder_h_dim_d = 64 # type=int
        self.d_learning_rate = 5e-4 # type=float
        self.d_steps = 2 # type=int        
        self.clipping_threshold_d = 0 # type=float  

        # Loss Options
        self.l2_loss_weight = 0 # type=float 
        self.best_k = 1 # type=int 

        # Output
        self.output_dir = os.getcwd()+ '/output/img_output'
        self.print_every = 5 # type=int
        self.checkpoint_every = 100 # type=int
        self.checkpoint_name = 'checkpoint_traffic' 
        self.checkpoint_start_from = None
        self.restore_from_checkpoint = 1 # type=int
        self.num_samples_check = 5000 # type=int        

        # Misc
        self.use_gpu = 1 # type=int
        self.timing = 0 # type=int
        self.gpu_num = "1" # type=str   


args = CreateArg() 

In [33]:
train_path = "/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769/769_frame_0.jpg"
img_path =os.path.split(train_path)[0]
img_path2 = os.path.split(os.path.split(train_path)[0])[0]
_path = os.path.splitext(train_path)[1]
print(img_path)
print(img_path2)
# print(_path)
# img_path2 = os.path.split(img_path)
# print(img_path2)

/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/769
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img


In [18]:
import torchvision.transforms as T
import logging
logger = logging.getLogger(__name__)
import sys
sys.path.append('/home/gpuadmin/dev/Trajectory_Prediction/traffino')

In [20]:


from loader_traffic import data_loader #####

transforms = T.Compose([T.ToTensor()]) # Normalize the pixels and convert to tensor.

logger.info("Initializing train dataset")

train_dset = data_loader(args, train_path) # train_dset은 TrajectoryDataset, train_loader는 DataLoader


# full_dataset = ImageDataset(
#     "C:/Users/NGN/dev/Traffino/TRAFFINO/traffino/datasets/waterloo/img_test/769/", 
#     transforms
#     )

logger.info("Initializing val dataset")
val_dset = data_loader(args, val_path)

['769_frame_175.jpg', '769_frame_551.jpg', '769_frame_455.jpg', '769_frame_117.jpg', '769_frame_158.jpg', '769_frame_569.jpg', '769_frame_326.jpg', '769_frame_467.jpg', '769_frame_491.jpg', '769_frame_249.jpg', '769_frame_603.jpg', '769_frame_473.jpg', '769_frame_604.jpg', '769_frame_691.jpg', '769_frame_524.jpg', '769_frame_321.jpg', '769_frame_699.jpg', '769_frame_453.jpg', '769_frame_343.jpg', '769_frame_238.jpg', '769_frame_82.jpg', '769_frame_614.jpg', '769_frame_122.jpg', '769_frame_408.jpg', '769_frame_351.jpg', '769_frame_18.jpg', '769_frame_647.jpg', '769_frame_584.jpg', '769_frame_217.jpg', '769_frame_39.jpg', '769_frame_108.jpg', '769_frame_661.jpg', '769_frame_578.jpg', '769_frame_528.jpg', '769_frame_277.jpg', '769_frame_421.jpg', '769_frame_172.jpg', '769_frame_225.jpg', '769_frame_447.jpg', '769_frame_304.jpg', '769_frame_412.jpg', '769_frame_81.jpg', '769_frame_0.jpg', '769_frame_333.jpg', '769_frame_273.jpg', '769_frame_643.jpg', '769_frame_47.jpg', '769_frame_570.jpg'

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [ ]:
f_a=image_encoder(batch_img_list_c)
print(f_a)

In [ ]:
1. For encoded_train_images, here is an example to extract image features from InceptionV3:

#load training set

train_imgs = load_text(os.path.join(text_folder, "Flickr_8k.trainImages.txt"))

# Get the InceptionV3 model trained on imagenet data

model = InceptionV3(weights='imagenet')

# Remove the last layer (output softmax layer) from the inception v3

model_new = Model(model.input, model.layers[-2].output)

# define the function

def get_cnn_feature(img_folder, img_name, model):

# Convert all the images to size 299x299 as expected by the

# inception v3 model

img = load_img(os.path.join(img_folder, img_name), target_size=(299,299))

# Convert PIL image to numpy array of 3-dimensions

x = img_to_array(img)

# Add one more dimension

x = np.expand_dims(x, axis=0)

# preprocess images using preprocess_input() from inception module

x = preprocess_input(x)

x = model.predict(x)

# reshape from (1, 2048) to (2048, )

x = np.reshape(x, x.shape[1])

return x

# extract features for training set:

encoded_training_images = dict()

for img in imgs_train:

encoded_training_images[img] = get_feature(img_folder, img, model_new)

with open(os.path.join(proc_folder, 'encoded_train_images.pkl'), 'wb') as f:

pk.dump(encoded_training_images,f)